In [27]:
import pandas as pd
# Assuming data is in a CSV file, with 'question' and 'label' as columns
df = pd.read_csv('book2.csv')
 

In [28]:
# Check class distribution
print(df['Category'].value_counts())
# Balance classes if needed
df = df.groupby('Category').sample(n=min(df['Category'].value_counts()), random_state=42)
# Shuffle the dataset
df = df.sample(frac=1, random_state=42)
 

Category
"Adversal Effects"       32
"Medical Information"    32
"Product Quality"        28
Name: count, dtype: int64


In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
def encode_data(tokenizer, questions, max_length):
    encoded = tokenizer.batch_encode_plus(
        questions,
        truncation=True,
        padding='longest',
        max_length=max_length,
        return_tensors='pt'  # return PyTorch tensors
    )
    return encoded["input_ids"], encoded["attention_mask"]


# Use an appropriate max_length 
input_ids, attention_mask = encode_data(tokenizer, df['review'].tolist(), max_length=128)


In [42]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=3)
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,    # number of training epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)
# Create the Trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=(input_ids, attention_mask, df['Category']),
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [43]:
# Load the test dataset and preprocess it in a similar way as the training set
test_df = pd.read_csv('book1.csv')
test_input_ids, test_attention_mask = encode_data(tokenizer, test_df['review'].tolist(), max_length=128)
# Evaluate the model
trainer.evaluate((test_input_ids, test_attention_mask, test_df['Category']))

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]